# Snowpark Connection

> Helps Manage Snowflake Connection

In [7]:
#| default_exp connection

In [8]:
#| hide
from nbdev.showdoc import *

In [9]:
#| export
import os
import logging
import yaml
import base64

from typing import Optional, Dict
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.exceptions import SnowparkSessionException
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

logging.getLogger('snowflake.snowpark').setLevel(logging.WARNING)

In [4]:
#|export
class AuthenticationError(Exception):
    """Custom exception for authentication errors."""
    pass

class SnowparkConnection:
    def __init__(self, connection_config: Optional[Dict[str, str]] = None, config_file: str = 'snowflake_config.yaml'):
        self.connection_config = connection_config or self.load_connection_config(config_file)
        self.session = self._get_active_or_new_session()

    def load_connection_config(self, yaml_file: str) -> Dict[str, str]:
        config = {}
        if os.path.isfile(yaml_file):
            try:
                with open(yaml_file, 'r') as file:
                    config = yaml.safe_load(file).get('snowflake', {})
            except FileNotFoundError:
                logging.warning(f"Configuration file '{yaml_file}' not found. Using environment variables.")
            except yaml.YAMLError as e:
                logging.error(f"Error parsing YAML file: {e}")
                raise
        
        # Fallback to environment variables if certain keys are missing
        env_vars = [
            'SNOWFLAKE_ACCOUNT', 'SNOWFLAKE_USER', 'SNOWFLAKE_ROLE', 
            'SNOWFLAKE_WAREHOUSE', 'SNOWFLAKE_DATABASE', 'SNOWFLAKE_SCHEMA', 
            'SNOWFLAKE_PRIVATE_KEY_PATH'
        ]
        
        for var in env_vars:
            key = var.lower().split('snowflake_')[1]
            if os.getenv(var) and key not in config:
                config[key] = os.getenv(var)
        
        return config
    
    def _get_active_or_new_session(self) -> Session:
        try:
            # First, try to get an active session
            session = get_active_session()
            logging.info("Using existing active Snowpark session.")
            return session
        except SnowparkSessionException:
            logging.info("No active session found. Attempting to create a new session.")
            return self._create_new_session()

    def _create_new_session(self) -> Session:
        try:
            if 'private_key_path' in self.connection_config:
                self._configure_key_pair_auth(self.connection_config)
            
            session = Session.builder.configs(self.connection_config).create()
            logging.info("Snowpark session created successfully.")
            return session
        except Exception as e:
            logging.error(f"Error creating Snowpark session: {e}")
            raise AuthenticationError(f"Failed to create session: {str(e)}")

    def _load_private_key(self, private_key_path: str) -> bytes:
        try:
            with open(private_key_path, "rb") as key_file:
                p_key = serialization.load_pem_private_key(
                    key_file.read(),
                    password=None,
                    backend=default_backend()
                )
            private_key_bytes = p_key.private_bytes(
                encoding=serialization.Encoding.DER,
                format=serialization.PrivateFormat.PKCS8,
                encryption_algorithm=serialization.NoEncryption()
            )
            logging.info("Private key loaded successfully.")
            return private_key_bytes
        except FileNotFoundError:
            logging.error(f"Private key file not found: {private_key_path}")
            raise AuthenticationError(f"Private key file not found: {private_key_path}")
        except Exception as e:
            logging.error(f"Error loading private key: {e}")
            raise AuthenticationError(f"Error loading private key: {str(e)}")

    def _configure_key_pair_auth(self, config: Dict[str, str]) -> None:
        try:
            private_key = self._load_private_key(config['private_key_path'])
            config['private_key'] = private_key
            config.pop('private_key_path', None)
        except Exception as e:
            logging.error(f"Error configuring key pair authentication: {e}")
            raise AuthenticationError(f"Key pair authentication configuration failed: {str(e)}")

    def get_session(self) -> Session:
        return self.session

    def close_session(self) -> None:
        try:
            self.session.close()
            logging.info("Snowpark session closed successfully.")
        except Exception as e:
            logging.error(f"Error closing Snowpark session: {e}")
            raise

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close_session()

In [5]:
#| skip

# 1. Username and Password
def username_password_example():
    config = {
        'user': os.getenv('SNOWFLAKE_USER', ''),
        'password': os.getenv('SNOWFLAKE_PASSWORD', ''),
        'account': os.getenv('SNOWFLAKE_ACCOUNT', ''),
        'database': 'CORTEX',
        'warehouse': 'CORTEX_WH',
        'schema': 'DEV',
        'role': 'CORTEX_USER_ROLE'
    }
    
    try:
        with SnowparkConnection(connection_config=config) as conn:
            session = conn.get_session()
            print(session.sql("SELECT CURRENT_WAREHOUSE(), CURRENT_DATABASE(), CURRENT_SCHEMA()").collect())
    except AuthenticationError as e:
        print(f"Authentication failed: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

username_password_example()

[Row(CURRENT_WAREHOUSE()='CORTEX_WH', CURRENT_DATABASE()='CORTEX', CURRENT_SCHEMA()='DEV')]


Example Setup Key Pair Authentication Setup

1. Generate a private key (run these commands in your local terminal, not in Snowflake)

```bash
openssl genrsa 2048 | openssl pkcs8 -topk8 -inform PEM -out rsa_key.p8 -nocrypt
# OR
openssl genrsa 2048 | openssl pkcs8 -topk8 -v2 des3 -inform PEM -out rsa_key.p8
openssl rsa -in rsa_key.p8 -pubout -out rsa_key.pub
```

2. Assign the public key to a Snowflake user
-- Replace 'YOUR_USERNAME' with the actual username

```sql
ALTER USER YOUR_USERNAME SET RSA_PUBLIC_KEY='<paste_public_key_here>';
```

3. Verify the key is set correctly

```sql
DESC USER YOUR_USERNAME;
```


In [6]:
#| skip
# Example usage
def key_pair_example():
    config = {
        'user': os.getenv('SNOWFLAKE_USER', ''),
        'account': os.getenv('SNOWFLAKE_ACCOUNT', ''),
        'private_key_file': os.path.expanduser('../rsa_key.p8'),  # or use 'private_key_path'
        'database': 'CORTEX',
        'warehouse': 'CORTEX_WH',
        'schema': 'DEV',
        'role': 'CORTEX_USER_ROLE'
    }
    
    try:
        with SnowparkConnection(connection_config=config) as conn:
            session = conn.get_session()
            print(session.sql("SELECT CURRENT_USER(), CURRENT_ROLE()").collect())
    except AuthenticationError as e:
        print(f"Authentication failed: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

key_pair_example()

[Row(CURRENT_USER()='JD_SERVICE_ACCOUNT_ADMIN', CURRENT_ROLE()='CORTEX_USER_ROLE')]


In [15]:
#| hide
import nbdev; nbdev.nbdev_export()